In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
userSchema = StructType() \
        .add("window",StructType()\
             .add("start",TimestampType())\
             .add("end",TimestampType()))\
        .add("system", StringType())\
        .add("count", LongType())

In [2]:
raw_data = spark\
.readStream.format("parquet")\
.schema(userSchema)\
.load("/cms/users/carizapo/ming/groupdata_cmsweb_logs");
raw_data.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- system: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
avg_data= raw_data.writeStream
    .trigger(Trigger.ProcessingTime("1 seconds"))
    .outputMode(OutputMode.Update())
    .format("exactlyonce.CustomSinkProvider")


In [3]:
# from pyspark.sql import Window
#function to calculate number of seconds from number of days
# days = lambda i: i * 86400
#create window by casting timestamp to long (number of seconds)
# w = (Window.orderBy(col("window.start").cast('long')).rangeBetween(-days(7), 0))

# avg_data = raw_data.withColumn('rolling_average', avg("count")\
#                                .over(w))

avg_data=raw_data\
.withWatermark("window.start", "1 minute")\
.groupBy("system")\
.agg(avg("count").alias("avg"))

avg_data.printSchema()

Py4JJavaError: An error occurred while calling o74.withWatermark.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: makeCopy, tree:
'EventTimeWatermark 'window.start, interval 1 minutes
+- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@5b71ba79,parquet,List(),Some(StructType(StructField(window,StructType(StructField(start,TimestampType,true), StructField(end,TimestampType,true)),true), StructField(system,StringType,true), StructField(count,LongType,true))),List(),None,Map(path -> /cms/users/carizapo/ming/groupdata_cmsweb_logs),None), FileSource[/cms/users/carizapo/ming/groupdata_cmsweb_logs], [window#0, system#1, count#2L]

	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.catalyst.trees.TreeNode.makeCopy(TreeNode.scala:377)
	at org.apache.spark.sql.catalyst.plans.QueryPlan.mapExpressions(QueryPlan.scala:128)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveReferences$$anonfun$apply$9.applyO

In [8]:
raw_data_flow = raw_data.writeStream.queryName("hdfs").outputMode("Append").format("memory").start()

In [145]:
avg_data_flow = avg_data.writeStream.queryName("avg").outputMode("Append").format("memory").start()

AnalysisException: 'Non-time-based windows are not supported on streaming DataFrames/Datasets;;\nWindow [collect_list(count#159L, 0, 0) windowspecdefinition(_w0#519L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, -2, currentrow$())) AS count_listing#517], [_w0#519L ASC NULLS FIRST]\n+- Project [window#157, system#158, count#159L, cast(window#157.start as bigint) AS _w0#519L]\n   +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@483a5412,parquet,List(),Some(StructType(StructField(window,StructType(StructField(start,TimestampType,true), StructField(end,TimestampType,true)),true), StructField(system,StringType,true), StructField(count,LongType,true))),List(),None,Map(path -> /cms/users/carizapo/ming/groupdata_cmsweb_logs),None), FileSource[/cms/users/carizapo/ming/groupdata_cmsweb_logs], [window#157, system#158, count#159L]\n'

In [5]:
spark.streams.active

In [46]:
# raw_data_flow.lastProgress
raw_data_flow.processAllAvailable()

In [12]:
raw_data_flow.stop()

In [9]:
alerts = spark.sql("select * from hdfs")
alerts.show()

+--------------------+--------------------+-----+
|              window|              system|count|
+--------------------+--------------------+-----+
|[2019-06-21 10:02...|/das/request?pid=...|    4|
|[2019-06-21 09:29...|/das/request?pid=...|    3|
|[2019-06-21 09:44...|/das/request?pid=...|    9|
|[2019-06-21 09:44...|/das/request?pid=...|    6|
|[2019-06-21 09:43...|/das/request?pid=...|    1|
|[2019-06-21 10:02...|/das/request?inst...|    1|
|[2019-06-21 09:28...|/das/request?inst...|    1|
|[2019-06-21 09:44...|/das/request?inst...|    1|
|[2019-06-21 09:44...|/das/request?inpu...|    1|
|[2019-06-21 09:43...|/das/request?inst...|    1|
|[2019-06-21 09:53...|/das/request?pid=...|    1|
|[2019-06-21 09:23...|/das/request?pid=...|    2|
|[2019-06-21 09:53...|/das/request?view...|    1|
|[2019-06-21 09:23...|/das/request?view...|    1|
|[2019-06-21 09:38...|/dqm/offline/data...|    1|
|[2019-06-21 09:38...|/dqm/offline/data...|    1|
|[2019-06-21 09:42...|/dqm/offline/data...|    1|
